# ** 2.Loading external stability and expression datasets ** 

**NESG Solubility** 
(https://loschmidt.chemi.muni.cz/soluprot/?page=download)
* 10k proteins
* Labels: exp, sol, uniprot id or local ID 
* Units: integer 

**Soluprot Solubility**
(https://loschmidt.chemi.muni.cz/soluprot/?page=download)
* 11k training, 3k test
* Label: solubility, number IDs with no conversion map (has seq)
* Unit: 0/1

**Price Solubility**
(https://pmc.ncbi.nlm.nih.gov/articles/PMC3372292/)
* 7k proteins 
* Label: usability. uniprot id
* Unit: 0/1

**PSI Solubility** 
(https://academic.oup.com/bioinformatics/article/36/18/4691/5860015?login=false)
* 11k proteins
* Label: solubility, Aa0000 ID scheme (has seq)
* Unit: 0/1
* Note: ecoli with custom IDs, dropped for now 

**Meltome Stability** 
(https://meltomeatlas.proteomics.wzw.tum.de/master_meltomeatlasapp/)
* 1M variants 
* Label: temperature, meltpoint, fold_change, uniprot id 
* Note: ecoli with custom IDs, dropped for now  

**FireprotDB Stability** 
(https://loschmidt.chemi.muni.cz/fireprotdb/)
* 53k variants
* Label: ddG, dTm, pH, Tm, mutation_effect, uniprot id 

**ThermomutDB Stability**
(https://biosig.lab.uq.edu.au/thermomutdb/downloads)
* 12k variants
* Label: pH, ddG, temperature, dTm, uniprot/pdb id 
* Note: these genes were not retrieved from the database due to removal from uniprotkb:  A0A410ZNC6 D0WVP7 G7LSK3 GQ884175 M5A5Y8 Q9REI6

**CAFA** 
(https://www.kaggle.com/competitions/cafa-5-protein-function-prediction/code)
* 142k variants

**Novozyme**
(https://www.kaggle.com/code/jinyuansun/eda-and-finetune-esm)
* 31k variants

**Protsol Solubility**
(https://huggingface.co/datasets/AI4Protein/ProtSolM)
* 71k proteins
* Label: solubility, no ID but has sequence
* Unit: 0/1 

**MaveDB** 
- https://mavedb.org/search?target-organism-name=Escherichia+coli+K-12 

**ProteinGym**
-  

**Align2023**
- there is 4 enzymes with train.csv datasets in each folder, they all have mutation codes, some are multi- so will need to only get the single mutation code ones. they all have the sequence we will only look at beta-glucosidaseB and alphaamylase. 

**Prothermdb** 
- sent the email for access 

In [ ]:
#Align2023 

In [ ]:
#we will now load "masterdb.csv" found under data
import pandas as pd 
import os 
path = "data/masterdb.tsv"
df = pd.read_csv(path,sep="\t")
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("(", "")
    .str.replace(")", "")
    .str.replace("?", "")
)
df2 = pd.DataFrame()
df2["id"]=df["name"]
df2["sequence"] = df["protein_sequence"].astype(str)
print(df2)

def dftofasta(df,outfile):
    with open(outfile,"w") as f:
        for index,row in df.iterrows():
            f.write(f">{row['id']}\n")
            f.write(f"{row['sequence']}\n")
    return outfile 

#dftofasta(df2,"data/masterdb.fasta")

In [ ]:
############################################
#setting the paths, loading them into dataframes, and making the merged fasta file to cdhit 
############################################


#PSI_PATH      = "data/benchmark/sol_benchmark/PSI_Biology_solubility_trainset.csv"
#psi_detail_path = "data/benchmark/sol_benchmark/PSI_all_data_esol.tab"
NESG_PATH     = "data/benchmark/sol_benchmark/nesg/nesg.csv"
nesg_fasta_path = "data/benchmark/sol_benchmark/nesg/nesg.fasta"
PRICE_PATH    = "data/benchmark/sol_benchmark/Price_usability_trainset.csv"
soluprot_train_path = "data/benchmark/sol_benchmark/soluprot_data/training_set.csv"
soluprot_test_path = "data/benchmark/sol_benchmark/soluprot_data/test_set.csv" 
soluprot_train_fasta = "data/benchmark/sol_benchmark/soluprot_data/training_set.fasta"
soluprot_test_fasta = "data/benchmark/sol_benchmark/soluprot_data/test_set.fasta" 
#meltome_path = "data/benchmark/stab_benchmark/meltome_cross-species.csv"
#meltome_fasta_path = "data/benchmark/stab_benchmark/meltome_fasta.fasta"
fireprot_path = "data/benchmark/stab_benchmark/fireprotdb_results_stability.csv"
thermomutdb_path = "data/benchmark/stab_benchmark/thermomutdb.json"
thermomutdb_fasta = "data/benchmark/stab_benchmark/thermomutdb.fasta"
protsol_train_path = "data/benchmark/protsolm_data/protsolm_train.csv"
protsol_test_path = "data/benchmark/protsolm_data/protsolm_test.csv"
novozyme_test_path = "data/benchmark/novozymes-enzyme-stability-prediction/test.csv"
novozyme_train_path = "data/benchmark/novozymes-enzyme-stability-prediction/train.csv"
novozyme_test_labels_path = "data/benchmark/novozymes-enzyme-stability-prediction/test_labels.csv"


#LOAD DATASETS 
#psi = load_psi("data/benchmark/sol_benchmark/PSI_Biology_solubility_trainset.csv","data/benchmark/sol_benchmark/PSI_all_data_esol.tab")
nesg = load_nesg(NESG_PATH, nesg_fasta_path) #no seq col 
price = load_price(PRICE_PATH) #fasta 
soluprot = load_soluprot(
    soluprot_train_path,
    soluprot_test_path,
    soluprot_train_fasta,
    soluprot_test_fasta    
)
fireprot = load_fireprot(fireprot_path) #sequence
thermomut = load_thermomut(thermomutdb_path,thermomutdb_fasta)
#meltome = load_meltome(meltome_path,meltome_fasta_path) 
protsolm = load_protsolm(protsol_train_path, protsol_test_path) #aa_seq
novozymes = load_novozymes(
    novozyme_train_path, novozyme_test_path, novozyme_test_labels_path
)

#ADD FASTA TO DATAFRAMES IF THE CSV DID NOT HAVE IT
nesg_fasta = read_fasta_dict(nesg_fasta_path)
nesg["sequence"] = nesg["id"].map(nesg_fasta)
price["sequence"] = price["fasta"]
protsolm["sequence"] = protsolm["aa_seq"]


#MERGE ALL FASTA FILES FOR CD-HIT
def fasta_merger_from_dfs(datasets, outpath):
    with open(outpath, "w") as out:
        for i, df in enumerate(datasets):
            for idx, row in df.iterrows():
                seq = row["sequence"]
                if seq is None or pd.isna(seq):
                    continue
                header = f"{df.__class__.__name__}_{i}_{idx}"
                out.write(f">{header}\n{seq}\n")

datasets = [nesg, price, soluprot, fireprot, protsolm,thermomut, novozymes]
fasta_merger_from_dfs(datasets, "allbenchmarks.fasta")

#SETTING  CANONICAL IDS
for i, df in enumerate(datasets):
    df["canonical_id"] = [
        f"{df.__class__.__name__}_{i}_{idx}" for idx in df.index
    ]

In [ ]:
############################################
#Setting up functions 
############################################
import pandas as pd
import json
import numpy as np
from pathlib import Path
import requests
from Bio.SeqIO.FastaIO import SimpleFastaParser

def read_fasta_dict(path: str):
    seqs = {}
    with open(path) as fh:
        for header, seq in SimpleFastaParser(fh):
            sid = header.split()[0].strip()
            seqs[sid] = seq.strip()
    return seqs

def load_nesg(csv_path: str, fasta_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)  # uses CSV header row directly: id, exp, sol
    seqs = read_fasta_dict(fasta_path)
    #"sid" "usability" "fasta" 
    df["sequence"] = df["id"].map(seqs)
    return df 

def load_psi(csv_path: str, psi_detail_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)                        # has sid + fasta + labels
    psi_all = pd.read_csv(psi_detail_path, sep="\t")  # extra metadata

    # merge on sid
    df = df.merge(psi_all, on="sid", how="left")

    # sequence is already in the "fasta" column
    df["sequence"] = df["fasta"]

    return df
def load_price(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)
    df["sequence"] = df["fasta"]
    # "sid" "usability" "fasta" 
    return df

def load_soluprot(train_csv: str, test_csv: str,
                  train_fasta_path: str, test_fasta_path: str) -> pd.DataFrame:

    # load FASTA → dict, keys exactly as in FASTA headers
    train_fasta = read_fasta_dict(train_fasta_path)
    test_fasta  = read_fasta_dict(test_fasta_path)

    # merge FASTA dicts
    fasta = {**train_fasta, **test_fasta}

    # load CSVs
    df1 = pd.read_csv(train_csv)
    df2 = pd.read_csv(test_csv)

    df = pd.concat([df1, df2], ignore_index=True)

    # map sequences using the exact ids
    df["sequence"] = df["sid"].map(fasta)


    return df


def load_meltome(csv_path: str, fasta_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)

    # load fasta into dict: {uniprot_id: sequence}
    fasta = read_fasta_dict(fasta_path)

    # extract uniprot prefix from Protein_ID (before "_")
    df["uniprot_id"] = df["Protein_ID"].astype(str).apply(lambda x: x.split("_")[0])

    # map sequences
    df["sequence"] = df["uniprot_id"].map(fasta)

    return df

def load_fireprot(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)
    # "uniprot_id" "pdb_id" "muutation" "ddG" "dTm" "pH" "tm" "mutation_effect" "sequence"
    return df 

def load_thermomut(json_path: str, fasta_path: str) -> pd.DataFrame:
    # load JSON metadata
    with open(json_path) as fh:
        data = json.load(fh)
    df = pd.DataFrame(data)

    # load FASTA sequences
    fasta_dict = read_fasta_dict(fasta_path)

    # map UniProt → sequence
    # JSON column is "uniprot"
    df["sequence"] = df["uniprot"].map(fasta_dict)

    # ensure required labels exist even if missing in JSON
    for col in ["ph","ddg","temperature","dtm","PDB_wild",
                "pdb_mutant","mutation_code","mutated chain","effect"]:
        if col not in df.columns:
            df[col] = None

    return df

def load_protsolm(train_csv: str, test_csv: str) -> pd.DataFrame:
    df1 = pd.read_csv(train_csv)
    df2 = pd.read_csv(test_csv)
    df = pd.concat([df1, df2], ignore_index=True)
    # "aa_seq" "detail"
    return df

def fasta_merger(fasta_paths: list, outpath: str):
    with open(outpath, "w") as out:
        for path in fasta_paths:
            with open(path) as fh:
                for line in fh:
                    out.write(line)

def parse_cd_hit_clusters(clstr_path):
    clusters = {}
    current = None

    with open(clstr_path) as f:
        for line in f:
            line = line.strip()
            if line.startswith(">Cluster"):
                current = line.split()[1]
                clusters[current] = []
            else:
                # Example: "0       50aa, >SEQ123... *"
                sid = line.split(">")[1].split("...")[0]
                clusters[current].append(sid)

    return clusters

def fasta_merger_from_dfs(datasets, outpath):
    with open(outpath, "w") as out:
        for i, df in enumerate(datasets):
            for idx, row in df.iterrows():
                seq = row["sequence"]
                if seq is None or pd.isna(seq):
                    continue
                header = f"{df.__class__.__name__}_{i}_{idx}"
                out.write(f">{header}\n{seq}\n")

def fetch_uniprot_fasta(ids, out_fasta, delay=0.15):
    """Fetch FASTA for many UniProt IDs with error handling."""
    with open(out_fasta, "w") as out:
        for uid in ids:
            url = f"https://rest.uniprot.org/uniprotkb/{uid}.fasta"
            r = requests.get(url, timeout=10)

            if r.status_code == 200 and r.text.startswith(">"):
                out.write(r.text.strip() + "\n")
            else:
                # write placeholder for failed fetch
                out.write(f">{uid}\nFAILED_FETCH\n")

            time.sleep(delay)  # rate-limit to avoid 500 errors



def load_novozymes(train_path: str, test_path: str, test_labels_path: str) -> pd.DataFrame:
    train_df = pd.read_csv(train_path)         # seq_id, protein_sequence, pH, Tm
    test_df = pd.read_csv(test_path)           # seq_id, protein_sequence, pH
    test_labels = pd.read_csv(test_labels_path)  # seq_id, Tm

    # merge test with its labels
    test_df = test_df.merge(test_labels, on="seq_id", how="left")

    # unify column names to match your other datasets
    train_df["sequence"] = train_df["protein_sequence"]
    test_df["sequence"]  = test_df["protein_sequence"]

    # combine train + test into one dataframe
    df = pd.concat([train_df, test_df], ignore_index=True)

    return df